## Index Api

index函数允许将来自任何来源的文档加载并保持同步到矢量存储中，可有效地避免将重复的内容写入向量数据库。

将文档索引到向量存储中时，可能会清理向量存储中的一些现有文档。在某些情况下，您可能希望清理与正在编辑索引的新文档派生自相同来源的任何现有文档。在其他情况下，您可能希望批量清理所有现有文档。index函数清理模式允许您选择所需的行为：
|模式	|内容去重	|并行处理	|删除源文档	|清理派生文档	|清理时机|特性|适用范围|
| --- | --- | --- | --- | --- | --- | --- | --- |
|None	|✅	|✅	|❌	|❌	|-| 允许重复内容|适合静态文档集|
|Incremental	|✅	|✅	|❌	|✅	|持续| 实时清理变更文档|适合频繁更新的文档|
|Full	|✅	|❌	|✅	|✅	|批次结束| 批次结束后彻底清理|适合定期全面更新|
|Scoped_Full	|✅	|✅	|❌	|✅	|批次结束| 类似 Full 模式,保留并行处理能力|精确清理当前批次相关文档|


In [35]:
from langchain.indexes import index, SQLRecordManager
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

import os
root_dir = os.path.abspath('..')

## 1. 准备embeddings及存储器

In [17]:
embeddings = OllamaEmbeddings(model="llama2-chinese")

vectorstore = Chroma(
    persist_directory=os.path.abspath(f"{root_dir}/databases/"),
    embedding_function=embeddings,
    collection_name="test_index"
)

namespace = f"chroma/test_index"
record_manager = SQLRecordManager(
    namespace=namespace,
    db_url="sqlite+pysqlite:///" + os.path.abspath(f"{root_dir}/databases/index.db")
)
# 初始化表结构
record_manager.create_schema()

## 2. 声明文档内容

In [18]:
doc1 = Document(page_content="A cute kitty", metadata={"source": "kitty.txt"})
doc2 = Document(page_content="A cute panda", metadata={"source": "panda.txt"})
doc3 = Document(page_content="A cute cat", metadata={"source": "cat.txt"})
doc4 = Document(page_content="A cute dog", metadata={"source": "dog.txt"})

def _clear():
    """利用完全删除模式的特性清理文档：未传递文档给索引，存在于向量存储中的所有文档都将被删除"""
    index([], record_manager, vectorstore, cleanup="full", source_id_key="source")

## 3.None模式（无删除模式）

In [51]:
_clear()
# 有重复文档
ir = index(
    [doc1, doc1, doc2],
    record_manager,
    vectorstore,
    cleanup=None,
    source_id_key="source"
)
print("有重复文档", ir)

# 三个不同文档
ir = index(
    [doc1, doc2, doc3],
    record_manager,
    vectorstore,
    cleanup=None,
    source_id_key="source"
)
print("三个不同文档", ir)

# 再次索引相同文档
ir = index(
    [doc1, doc2, doc3],
    record_manager,
    vectorstore,
    cleanup=None,
    source_id_key="source"
)
print("再次索引相同文档", ir)

# 不提供文档
ir = index(
    [],
    record_manager,
    vectorstore,
    cleanup=None,
    source_id_key="source"
)
print("不提供文档", ir)

# 使用增量删除模式，只删除变量的旧版本，保留未变量的文档
doc2.page_content = "A heavy and cute panda"
ir = index(
    [doc2],
    record_manager,
    vectorstore,
    cleanup=None,
    source_id_key="source"
)
print("文档变更", ir)

有重复文档 {'num_added': 2, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}
三个不同文档 {'num_added': 1, 'num_updated': 0, 'num_skipped': 2, 'num_deleted': 0}
再次索引相同文档 {'num_added': 0, 'num_updated': 0, 'num_skipped': 3, 'num_deleted': 0}
不提供文档 {'num_added': 0, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}
文档变更 {'num_added': 1, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}


## 4. 增量模式

In [ ]:
_clear()

doc2.page_content = "A cute panda"
# 有重复文档
ir = index(
    [doc1, doc1, doc2],
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source"
)
print("有重复文档", ir)

# 再次索引相同文档
ir = index(
    [doc1, doc2],
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source"
)
print("再次索引相同文档", ir)

# 使用增量删除模式，只删除变量的旧版本，保留未变量的文档
doc2.page_content = "A heavy and cute panda"
ir = index(
    [doc1, doc2],
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source"
)
print("文档变更", ir)

# 如果不提供文档，则不会有变化
ir = index(
    [],
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source"
)
print("不提供文档", ir)

有重复文档 {'num_added': 2, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}
再次索引相同文档 {'num_added': 0, 'num_updated': 0, 'num_skipped': 2, 'num_deleted': 0}
文档变更 {'num_added': 1, 'num_updated': 0, 'num_skipped': 1, 'num_deleted': 1}
不提供文档 {'num_added': 0, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}


## 5. 全量模式

In [50]:
_clear()

doc2.page_content = "A cute panda"

# 新库索引
ir = index(
    [doc1, doc2],
    record_manager,
    vectorstore,
    cleanup="full",
    source_id_key="source"
)
print("新库索引", ir)

# 索引doc3，doc1和doc2会被删除
ir = index(
    [doc3],
    record_manager,
    vectorstore,
    cleanup="full",
    source_id_key="source"
)
print("索引doc3", ir)

新库索引 {'num_added': 2, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}
索引doc3 {'num_added': 1, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 2}
